<a href="https://colab.research.google.com/github/LarrySnyder/stockpyl/blob/master/notebooks/Stockpyl_Tutorial_%C2%A71_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stockpyl Tutorial
=================

(This notebook is a companion to Snyder, L. V., "[Stockpyl: A Python Package for Inventory Optimization and Simulation](https://pubsonline.informs.org/doi/10.1287/educ.2023.0256)," in: Bish, E. K. and H. Balasubramanian, _INFORMS TutORials in Operations Research_, 156–197, 2023.)



# Section 1: Introduction


## 1.1 About Stockpyl

Stockpyl is a Python package for inventory optimization and simulation. It implements classical single-node inventory models like the economic order quantity (EOQ), newsvendor, and Wagner–Whitin problems. It also contains algorithms for multi-echelon inventory optimization (MEIO) under both stochastic-service model (SSM) and guaranteed-service model (GSM) assumptions. Moreover, it has extensive features for simulating multi-echelon inventory systems.

In these notebooks and the accompanying TutORial, we provide an overview of the inventory-optimization features of Stockpyl. Stockpyl also has simulation features, but they are outside the scope of the TutORial and will be discussed in a future document.


> **Remark:** The notation and references (equations, sections, examples, etc.) used throughout this tutorial refer to Snyder and Shen, _[Fundamentals of Supply Chain Theory](https://coral.ise.lehigh.edu/sctheory/)_, 2nd edition (2019). We will use the shorthand _FoSCT_ to refer to this text.

Stockpyl

For more info:
* `stockpyl` package on [PyPI](https://pypi.org/project/stockpyl/)
* `stockpyl` documentation on [Read the Docs](https://stockpyl.readthedocs.io/en/latest/index.html)
* `stockpyl` source code on [GitHub](https://github.com/LarrySnyder/stockpyl)

Feedback, suggestions, feature requests, and bug reports about Stockpyl are always welcome. The preferred method is to post an issue on the Stockpyl GitHub repo at https://github.com/LarrySnyder/stockpyl/issues. Alternatively, feel free to email me.

If you wish to contribute to the Stockpyl project, you can browse the outstanding issues at the link above, or develop code to address an aspect you are interested in. Once your code is ready for review, please submit a pull request at https://github.com/LarrySnyder/stockpyl/pulls. Thank you for your contributions.

## 1.2 A Few Quick Examples

First, we need to install the Stockpyl package locally. (If you are installing `stockpyl` locally, not in a Google Colab notebook, delete the exclamation point (`!`).)

In [ ]:
!pip install stockpyl

Next, import the package as usual:

In [ ]:
import stockpyl

Stockpyl is organized into [modules](https://stockpyl.readthedocs.io/en/latest/api/api.html), each of which contains code for a particular aspect of inventory optimization or simulation:

* Single-Echelon Inventory Optimization
    * `eoq` Module
    * `newsvendor` Module
    * `rq` Module
    * `ss` Module
    * `wagner_whitin` Module
    * `finite_horizon` Module
    * `supply_uncertainty` Module
* Multi-Echelon Inventory Optimization
    * `ssm_serial` Module
    * `gsm_tree` Module
    * `gsm_serial` Module
    * `gsm_helpers` Module
    * `meio_general` Module
* Simulation
    * `sim` Module
    * `sim_io` Module
* Data Types
    * `supply_chain_node` Module
    * `supply_chain_network` Module
    * `demand_source` Module
    * `policy` Module
    * `disruption_process` Module
* Other Modules
    * `instances` Module
    * `loss_functions` Module
    * `optimization` Module
    * `helpers` Module

This tutorial will mostly discuss the Single- and Multi-Echelon Inventory Optimization modules.

---
First up: Solve the EOQ problem with a fixed cost of 8, a holding cost of 0.225, and a demand rate of 1300 (Example 3.1 in _FoSCT_):

In [ ]:
from stockpyl.eoq import economic_order_quantity
Q, cost = economic_order_quantity(fixed_cost=8, holding_cost=0.225, demand_rate=1300)

Display the values returned:

In [ ]:
Q, cost

Solve the newsvendor problem with a holding cost of 0.18, a stockout cost of 0.70, and demand that is normally distributed with mean 50 and standard deviation 8 (Example 4.3 in _FoSCT_):


In [ ]:
from stockpyl.newsvendor import newsvendor_normal
S, cost = newsvendor_normal(holding_cost=0.18, stockout_cost=0.70, demand_mean=50, demand_sd=8)

In [ ]:
S, cost

> **Remark:** Most functions in Stockpyl use longer, more descriptive parameter names (`holding_cost`, `fixed_cost`, etc.) rather than the shorter notation assigned to them in textbooks and articles ($h$, $K$, etc.).

Stockpyl can solve the Wagner–Whitin model using dynamic programming (DP):

In [ ]:
from stockpyl.wagner_whitin import wagner_whitin
T = 4
h = 2
K = 500
d = [90, 120, 80, 70]
Q, cost, theta, s = wagner_whitin(T, h, K, d)

In [ ]:
Q # Optimal order quantities

In [ ]:
cost # Optimal cost

In [ ]:
theta # Cost-to-go function

In [ ]:
s # Optimal next period to order in

And finite-horizon, stochastic inventory optimization models, also via DP:

In [ ]:
from stockpyl.finite_horizon import finite_horizon_dp
T = 5
h = 1
p = 20
h_terminal = 1
p_terminal = 20
c = 2
K = 50
mu = 100
sigma = 20
s, S, cost, _, _, _ = finite_horizon_dp(T, h, p, h_terminal, p_terminal,
    c, K, mu, sigma)

In [ ]:
s # Reorder points

In [ ]:
S # Order-up-to levels

Stockpyl includes an implementation of the Clark and Scarf (1960) algorithm for stochastic serial systems (or, more precisely, Chen and Zheng's (1994) reworking of it:

In [ ]:
from stockpyl.supply_chain_network import serial_system
from stockpyl.ssm_serial import optimize_base_stock_levels
# Build network.
network = serial_system(
        num_nodes=3,
        node_order_in_system=[3, 2, 1],
        echelon_holding_cost=[4, 3, 1],
        local_holding_cost=[4, 7, 8],
        shipment_lead_time=[1, 1, 2],
        stockout_cost=40,
        demand_type='N',
        mean=10,
        standard_deviation=2
    )
# Optimize echelon base-stock levels.
S_star, C_star = optimize_base_stock_levels(network=network)

In [ ]:
S_star # Optimal base-stock levels.

In [ ]:
C_star # Optimal expected cost.

Stockpyl has extensive features for simulating multi-echelon inventory systems. The code below simulates the same serial system as above, obtaining an average cost per period that is similar to what the theoretical model predicted.

In [ ]:
from stockpyl.supply_chain_network import echelon_to_local_base_stock_levels
from stockpyl.sim import simulation
from stockpyl.policy import Policy
# Convert to local base-stock levels and set nodes' inventory policies.
S_star_local = echelon_to_local_base_stock_levels(network, S_star)
for n in network.nodes:
	     n.inventory_policy = Policy(type='BS',
    	     base_stock_level=S_star_local[n.index], node=n)
# Simulate the system.
T = 1000
total_cost = simulation(network=network, num_periods=T, rand_seed=42)

In [ ]:
total_cost / T # average total cost per period (compare to 227.15 above).

Stockpyl also implements Graves and Willems's (2000) dynamic programming algorithm for optimizing committed service times (CSTs) in acyclical guaranteed-service model (GSM) systems:

In [ ]:
from stockpyl.gsm_tree import optimize_committed_service_times
from stockpyl.instances import load_instance
# Load a named instance, Example 6.5 from FoSCT.
tree = load_instance("example_6_5")
# Optimize committed service times.
opt_cst, opt_cost = optimize_committed_service_times(tree)

In [ ]:
opt_cst # Optimal committed service times.

In [ ]:
opt_cost # Optimal expected cost.

**Next Up:** Stockpyl Tutorial §2: Single-Echelon Inventory Optimization